# Sistemas de Recomendação em Python

Neste exercício será implementado um sistema de recomendação em Python seguindo o tutorial disponibilizado em : https://www.datacamp.com/community/tutorials/recommender-systems-python 

No notebook vamos ver como construir um simples modelo de sistemas de recomendação, assim como um modelo de sistemas de recomendação baseado em conteúdo.

Vamos classificar os sistemas de recomendação em 3 tipos:

* <b>Recomendadores simples:</b> Faz uma recomendação generalizada para todos os usuários, ela se baseia em popularidade ou gênero do filme. A idéia básica por trás desse sistema é que os filmes mais populares vão ter mais chances de serem aceitos para o público. O top 250 do IMDB é um exemplo desse sistema.
* <b>Recomendadores baseado em conteúdo:</b> Ela sugere filmes similares baseados em um filme em particular. Esse sistema usa o metadata do filme para fazer as recomendações. A idéia geral desse sistema é de que a pessoa que gostou de um certo filme vai gostar de outros filmes similares.
* <b>Mecanismos de filtragem colaborativa:</b> Esses sistemas tentam prever a nota que o usuário vai dar para um filme baseado em notas de outros usuários. Diferente dos recomendadores baseado em conteúdo, os filtros colaborativos não precisam do metadado do filme.

Agora vamos construir esses 3 tipos de sistemas de recomendação:

## Recomendadores simples

Nesta seção vamos construir um clone simplificado do top 250 filmes do IMDB usando o dataset coletado do IMDB.

Link para o dataset do IMDB: https://www.kaggle.com/rounakbanik/the-movies-dataset/data

Vamos ter que realizar os seguintes passos:

* Decidir a pontuação para classificar os filmes.
* Calcular a nota para todos os filmes.
* Ordenar os filmes baseado nas notas e mostrar aqueles que estão no topo.

Antes de começar qualquer coisa, vamos carregar o dataset dos filmes no pandas DataFrame:

In [1]:
# Import Pandas
import pandas as pd

# Load Movies Metadata
metadata = pd.read_csv('./movie_dataset/movies_metadata.csv', low_memory=False)

# Print the first three rows
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Agora vamos ordernar os filmes a partir de sua avaliação, porém para fazer isso vamos utilizar a Weighted Rating(WR), com a WR será possível diferenciar 2 filmes que tiveram a mesma avaliação mas com números diferentes de votos, isto é, a classificação de um filme com avaliação 9 com 100000 votos vale mais que um filme com avaliação 9 com 10 votos. A formula da WR é descrita como:

$$WR =\left(\frac{v}{v+m}\times R \right)+\left(\frac{m}{v+m}\times C \right)$$

onde,
* <b>v</b> é o número de votos de um filme;
* <b>m</b> é a quantidade de votos mínimos necessários para o filme aparecer na lista;
* <b>R</b> é nota média do filme; e
* <b>C</b> é a média dos votos de todo o documento

Já sabemos <b>v</b> e <b>R</b> de todos os filmes pelo dataset. Também é possível calcular o <b>C</b> por esses dados.

In [2]:
# Calculate C
C = metadata['vote_average'].mean()
print(C)

5.618207215133889


Não existe um valor correto para <b>m</b>, seu valor depende de como o desenvolvedor vai determinar a quantidade certa de votos relevantes para a pesquisa.

Neste caso, vamos utilizar 90% como o limite, isto é, para o filme estar na tabela ele precisa ter mais votos do que pelo menos 90% dos filmes na lista, ele tem que estar entre os top 10%.

In [3]:
# Calculate the minimum number of votes required to be in the chart, m
m = metadata['vote_count'].quantile(0.90)
print(m)

160.0


Agora vamos criar um novo metadata com os filmes que respeitem o número mínimo de votos para estarem na lista:

In [4]:
# Filter out all qualified movies into a new DataFrame
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

Com o <b>.copy()</b>, nós garantimos que o novo <b>q_movies</b> dataframe é idependente do metadata original. Assim toda mudança feita no <b>q_movies</b> não afeta o metadata original.

Agora para ordenar os filmes vai ser necessário criar uma nova variável chamada nota(score). Com o <b>WR</b> vamos definir o <b>score</b> e criar uma nova classificação para os 4555 filmes que passaram da quantidade mínima de votos.

In [5]:
# Function that computes the weighted rating of each movie
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * C)

In [6]:
# Define a new feature 'score' and calculate its value with `weighted_rating()`
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

Vamos ordenar os filmes a partir do <b>score</b>, criando um top15 dos filmes.

In [7]:
#Sort movies based on score calculated above
q_movies = q_movies.sort_values('score', ascending=False)

#Print the top 15 movies
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(15)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.445869
834,The Godfather,6024.0,8.5,8.425439
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,8.421453
12481,The Dark Knight,12269.0,8.3,8.265477
2843,Fight Club,9678.0,8.3,8.256385
292,Pulp Fiction,8670.0,8.3,8.251406
522,Schindler's List,4436.0,8.3,8.206639
23673,Whiplash,4376.0,8.3,8.205404
5481,Spirited Away,3968.0,8.3,8.196055
2211,Life Is Beautiful,3643.0,8.3,8.187171


## Recomendador baseado em conteúdo em Python

### Recomendador baseado na descrição da sinopse

Nesta seção, vamos construir um sistema que recomenda filmes que são similares com outro filme. Mais especificamente, vamos calcular notas de similaridades entre os filmes baseado na sinopse e recomendar filmes baseado nessa nota de similaridade.

A descrição da sinopse está disponível como <b>overview</b> no metadata dos filmes.

In [8]:
#Print plot overviews of the first 5 movies.
metadata['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

Vamos computar vetores de Term Frequency-Inverse Document Frequency(TF-IDF) para cada documento. Com isso vamos ter uma matriz onde cada coluna representa uma palavra no vocabulário do <b>overview</b>, que são todas as palavras que aparecem pelo menos uma vez, e cada coluna representa um filme.

Na essência a nota TF-IDF é a frequência das palavras que aparecem no documento, o peso de uma palavra vai diminuir de acordo com a quantidade de vezes que ela aparece no documento. Isso é feito para reduzir a importância de palavras que aparecem com uma certa frequência no <b>overview</b>.

Felizmente, scikit-learn tem uma classe TfIdfVectorizer que produz a matriz TF-IDF.

In [9]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
metadata['overview'] = metadata['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(metadata['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(45466, 75827)

A partir disso, podemos ver que mais de 75000 de palavras diferentes foram utilizadas para descrever os mais de 45000 filmes no dataset.

Com essa matriz, podemos calcular uma nota de similaridade. Temos muitas maneiras de calcular a nota, como pela escala de distância euclidiana, pela escala de correlação de Pearson e pela similaridade do cosseno.

Neste documento, vamos usar a similaridade do cosseno para calcular a quantidade numérica que mostra a similaridade entre dois filmes. A similaridade do cosseno é definido como:
$$cosine \left(x,y\right) = \frac{x.y^T}{||x||.||y||}$$
Já que nós utilizamos o TF-IDF vectorizer, calcular o produto escalar vai diretamente nos dar a nota da similaridade do cosseno.
Portanto, vamos usar <b>linear_kernel()</b> do <b>sklearn</b> ao invés da similaridade do cosseno já que é mais rápido.

### Código que deveria ser executado mas não vamos fazer
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Aqui vamos nos divergir do tutorial

Se o código acima fosse executado, vai ocorrer um erro de memória porque a tfidf_matrix é muito grande, ela é uma matriz de mais de 45000 linhas.

Por isso para ser possível executar o código é necessário uma quantidade menor de filmes para ser analisado, para isso vamos utilizar o <b>q_movies</b>, ele, como foi visto na parte de recomendadores simples, é um metadata dos filmes só com os top 10% dos filmes. Desse modo teremos uma quantidade menor de filmes para analisar, possibilitando que o programa rode sem ocorrer o problema de falta de memória.

In [10]:
q_movies = metadata.copy().loc[metadata['vote_count'] >= m]
q_movies.shape

(4555, 24)

Observe que antes nossa quantidade de filmes era de 45466, e agora é de 4555.

In [11]:
#Print plot overviews of the first 5 movies.
q_movies['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
4    Just when George Banks has recovered from his ...
5    Obsessive master thief, Neil McCauley leads a ...
8    International action superstar Jean Claude Van...
Name: overview, dtype: object

Vamos resetar os indíces do nosso novo e menor dataframe:

In [12]:
q_movies = q_movies.reset_index()
q_movies['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    Just when George Banks has recovered from his ...
3    Obsessive master thief, Neil McCauley leads a ...
4    International action superstar Jean Claude Van...
Name: overview, dtype: object

Agora vamos repetir tudo que tinhamos feito com o metadata no q_movies:

In [13]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
q_movies['overview'] = q_movies['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(q_movies['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(4555, 19694)

In [14]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

Agora que temos uma tfidf_matrix menor, foi possível executar o código acima.

Vamos criar uma função que utiliza o nome de um filme como entrada e retorna uma lista dos 10 filmes mais parecidos. Para isso vamos precisar de um mecanismo que identifique o indíce de um filme no dataframe, dado o seu título.

In [15]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(q_movies.index, index=q_movies['title']).drop_duplicates()

Agora estamos prontos para relizar nossa recomendação. Estes são os passos que temos que seguir:
* Pegar o indíce do filme dado o seu título.
* Pegar a lista das notas da similaridade do cosseno de um filme em particular com todos os outros filmes. Converter isso em uma lista de tuplas onde o primeiro elemento é a posição e o segundo é a nota de similaridade.
* Ordenar a lista de tupla baseada nas notas de similaridades
* Pegar o top10 elementos da lista. Ignore o primeiro elemento porque ele refere ao próprio filme analisado.
* Retorne os títulos correspondentes aos indíces dos elementos do topo.

In [16]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return q_movies['title'].iloc[movie_indices]

Vamos executar alguns exemplos:

In [17]:
get_recommendations('The Dark Knight Rises')

47                               Batman Forever
2398                            The Dark Knight
444                              Batman Returns
185                                      Batman
2824                 Batman: Under the Red Hood
3084                           Batman: Year One
3318    Batman: The Dark Knight Returns, Part 1
4249                          Batman: Bad Blood
3355    Batman: The Dark Knight Returns, Part 2
981                Batman: Mask of the Phantasm
Name: title, dtype: object

In [18]:
get_recommendations('The Godfather')

354      The Godfather: Part II
3662                 Blood Ties
630     The Godfather: Part III
4437              Live by Night
3290                   Sinister
3262      The Cold Light of Day
3632                        Joe
1414          Road to Perdition
2268         Death at a Funeral
670           The Addams Family
Name: title, dtype: object

Apesar de que o sistema funcione, vemos que a qualidade das recomendações não é muito boa. "The Dark Knight Rises" retorna todos os filmes do Batman, no entanto é mais provável da pessoa que gostou do filme gostar mais dos outros filmes do Christopher Nolan. Isto é algo que não pode ser capturado pelo sistema atual.

## Recomendador baseado em créditos, gêneros e palavras-chave.

Para melhorar a qualidade do nosso recomendador, vamos usar mais metadata. Nesta seção, vamos construir um recomendador baseado nas seguintes metadata: os top3 atores, o diretor, gêneros relacionados e as palavras-chave dos filmes.

Os dados das palavras-chave, do elenco e da equipe não estão disponíveis no nosso atual dataset, então o nosso primeiro passo vai ser carregar e juntar eles no nosso dataframe principal.

OBS: Lembrando de nosso dataframe é o <b>q_movies</b>.
Por causa disso vamos remover a linha: "metadata = metadata.drop([19730, 29503, 35587])", que aparece no tutorial pois ela não é necessária porque já eliminamos esses IDs ruins quando começamos a usar o dataframe <b>q_movies</b>.

In [19]:
# Load keywords and credits
credits = pd.read_csv('./movie_dataset/credits.csv')
keywords = pd.read_csv('./movie_dataset/keywords.csv')

# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
q_movies['id'] = q_movies['id'].astype('int')

# Merge keywords and credits into your main q_movies dataframe
q_movies = q_movies.merge(credits, on='id')
q_movies = q_movies.merge(keywords, on='id')

In [20]:
# Print the first two movies of your newly merged q_movies
q_movies.head(2)

,index,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1..."


Com as nossas novas características, elenco, equipe e palavras-chave, vamos extrair os 3 mais importantes atores, o diretor e as palavras-chave associadas com aquele filme. Temos que converter essas características de uma forma que seja útil para nós

In [21]:
# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    q_movies[feature] = q_movies[feature].apply(literal_eval)

Agora vamos escrever funções que vão ajudar na extração das informações desejadas de cada característica. Primeiro, vamos importar o pacote NumPy para ter acesso às constantes NaN. Em seguida, podemos usá-las para escrever a função <b>get_director</b>().

In [22]:
# Import Numpy 
import numpy as np

In [23]:
# Get the director's name from the crew feature. If director is not listed, return NaN
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [24]:
# Returns the list top 3 elements or entire list; whichever is more.
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [25]:
# Define new director, cast, genres and keywords features that are in a suitable form.
q_movies['director'] = q_movies['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    q_movies[feature] = q_movies[feature].apply(get_list)

In [26]:
# Print the new features of the first 3 films
q_movies[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]","[Animation, Comedy, Family]"
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...","[Adventure, Fantasy, Family]"
2,Father of the Bride Part II,"[Steve Martin, Diane Keaton, Martin Short]",Charles Shyer,"[baby, midlife crisis, confidence]",[Comedy]


O próximo passo será coverter os nomes e instâncias de palavra-chave para minúsculo e retirar todos os espaços entre eles. Isto é feito para que seu vetorizador não considere o Johnny de "Johnny Depp" e "Johnny Galecki" como os mesmos. Depois desse passo, os atores citados acima serão representados como "johnnydepp" e "johnnygalecki" e vão ser diferentes no vetorizador. 

In [27]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [28]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    q_movies[feature] = q_movies[feature].apply(clean_data)

Agora podemos criar "metadata soup", no qual é uma string que contém todos os metadata que nós queremos para alimentar nosso vetorizador(chamados de atores, diretores e palavras-chave),

In [29]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [30]:
# Create a new soup feature
q_movies['soup'] = q_movies.apply(create_soup, axis=1)

Os próximos passos são os mesmo que a gente fez no recomendador baseado na descrição da sinopse. Uma diferença importante é que vamos usar <b>CountVectorizer()</b> ao invés de TF-IDF. Isso é porque não importa se tem atores que atuaram em mais filmes ou diretores que dirigiram mais filmes.

In [31]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(q_movies['soup'])

In [32]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [33]:
# Reset index of your main DataFrame and construct reverse mapping as before
q_movies = q_movies.reset_index()
indices = pd.Series(q_movies.index, index=q_movies['title'])

Agora podemos reutilizar nossa função <b>get_recommendations()</b> passando na nova matriz <b>cosine_sim2</b> como seu segundo argumento.

In [34]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

2411            The Dark Knight
1956              Batman Begins
2176               The Prestige
4014    Kidnapping Mr. Heineken
2930                     Faster
3026                     Takers
3102                 The Double
1467          Escape to Victory
1783                   Catwoman
2140                      Chaos
Name: title, dtype: object

In [35]:
get_recommendations('The Godfather', cosine_sim2)

630    The Godfather: Part III
354     The Godfather: Part II
341             Apocalypse Now
3                         Heat
137              Carlito's Way
309        Glengarry Glen Ross
467              Donnie Brasco
516              The Rainmaker
573          On the Waterfront
574            West Side Story
Name: title, dtype: object

Agora podemos observar uma lista de recomendações mais diversificada.

## Mecanismos de filtragem colaborativa

No tutorial que nos baseamos não foi implementado essa parte pois já estamos familiarizados com as idéias necessárias pra implementá-la.

# Conclusão

Com isso concluímos o tutorial de sistemas de recomendação em python.